<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px;" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png" height="50px"> <a href='https://harvard-iacs.github.io/2021-AC215/' target='_blank'><strong><font color="#A41034">AC215: Advanced Practical Data Science, MLOps</font></strong></a></h1>

# **<font color="#A41034">Exercise 1 - Dask</font>**

**Harvard University**<br/>
**Fall 2021**<br/>
**Instructor:**
Pavlos Protopapas<br/>

<hr style="height:2pt">

## **<font color="#A41034">Setup Notebook</font>**

**Copy & setup Colab**

1) Select "File" menu and pick "Save a copy in Drive"

**Installs**

In [3]:
!pip install dask dask[dataframe] dask-image

zsh:1: no matches found: dask[dataframe]


**Imports**

In [2]:
import os
import requests
import zipfile
import tarfile
import shutil
import math
import json
import time
import sys
import numpy as np
import pandas as pd

# Dask
import dask
import dask.dataframe as dd
import dask.array as da
from dask.diagnostics import ProgressBar

**Utils**

Here are some util functions that we will be using for this exercise

In [4]:
def download_file(packet_url, base_path="", extract=False, headers=None):
  if base_path != "":
    if not os.path.exists(base_path):
      os.mkdir(base_path)
  packet_file = os.path.basename(packet_url)
  with requests.get(packet_url, stream=True, headers=headers) as r:
      r.raise_for_status()
      with open(os.path.join(base_path,packet_file), 'wb') as f:
          for chunk in r.iter_content(chunk_size=8192):
              f.write(chunk)
  
  if extract:
    if packet_file.endswith(".zip"):
      with zipfile.ZipFile(os.path.join(base_path,packet_file)) as zfile:
        zfile.extractall(base_path)
    else:
      packet_name = packet_file.split('.')[0]
      with tarfile.open(os.path.join(base_path,packet_file)) as tfile:
        tfile.extractall(base_path)

## **<font color="#A41034">Dataset</font>**

### **Load Data**

In [5]:
start_time = time.time()
download_file("https://github.com/dlops-io/datasets/releases/download/v1.0/Parking_Violations_Issued_-_Fiscal_Year_2017.csv.zip", base_path="datasets", extract=True)
execution_time = (time.time() - start_time)/60.0
print("Download execution time (mins)",execution_time)

Download execution time (mins) 1.0878633499145507


In [6]:
parking_violation_csv = os.path.join("datasets","Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

## Q1: Compute Pi with a Slowly Converging Series

Leibniz published one of the oldest known series in 1676.  While this is easy to understand and derive, it converges very slowly.
https://en.wikipedia.org/wiki/Leibniz_formula_for_%CF%80 <br/>
$$\frac{\pi}{4} = 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} ...$$

While this is a genuinely cruel way to compute the value of $\pi$, it’s a fun opportunity to use brute force on a problem instead of thinking.
Compute using at least four billion terms in this sequence. Compare your time taken with numpy and dask.  On my mac, with numpy this took 44 seconds and with dask it took 5.7 seconds. 

*Hint:* Use dask array

**Checking  1e9 * 4  terms with numpy**

If 1e9 * 4 fails, try 1e9 * 2 or increase memory 

In [27]:
start_time = time.time()
positive_terms = np.sum(1/np.arange(1, 1e9 * 8 , 4))
negative_terms = np.sum(-1/np.arange(3, 1e9 * 8 , 4))
pi_computed = (np.sum(positive_terms) + np.sum(negative_terms)) * 4
execution_time = (time.time() - start_time)
print('executaion_time(S):' , execution_time)


executaion_time(S): 399.8725597858429


In [22]:
np.arange(3, 1e9 * 4 , 4)

array([3.00000000e+00, 7.00000000e+00, 1.10000000e+01, ...,
       3.99999999e+09, 4.00000000e+09, 4.00000000e+09])

In [28]:
# Error 
error = np.abs(pi_computed-np.pi)

# Report Results
print(f'Pi real value = {np.pi:14.12f}')
print(f'Pi computed value = {pi_computed:14.12f}')
print(f'Error = {error:6.3e}')

Pi real value = 3.141592653590
Pi computed value = 3.141592653339
Error = 2.504e-10


**Checking  1e9 * 4  terms with Dask**

In [29]:
# Your code here
start_time = time.time()
positive_sum = 1/da.arange(1, 1e9 * 8 , 4, chunks=1e7)
negative_sum = -1/da.arange(3, 1e9 * 8 , 4, chunks=1e7)
pi_computed = (da.sum(positive_sum) + da.sum(negative_sum)) * 4
step3_pi = pi_computed.compute()
execution_time = (time.time() - start_time)
print(execution_time)

19.281399965286255


In [30]:
error = np.abs(step3_pi - np.pi)

# Report Results
print(f'Pi real value = {np.pi:14.12f}')
print(f'Pi computed value = {step3_pi:14.12f}')
print(f'Error = {error:6.3e}')

Pi real value = 3.141592653590
Pi computed value = 3.141592653340
Error = 2.500e-10


## Filter Parking Tickets Dataset

According to the parking tickets data set documentation, the column called ‘Plate Type’ consists mainly of two different types, ‘PAS’ and ‘COM’; presumably for passenger and commercial vehicles, respectively. Maybe the rest are the famous parking tickets from the UN diplomats, who take advantage of diplomatic immunity not to pay their fines.

Create a filtered Dask DataFrame with only the commercial plates.
Persist it, so it is available in memory for future computations. Count the number of summonses in 2017 (i.e., Issue Year in 2016, 2017) issued to commercial plate types. Compute them as a percentage of the total data set. 

*Hint*: This is easy; it is only about 5-7 lines of code.

In [19]:
dict_1 = {'Summons Number': 'int64', 'Plate ID': 'object', 'Registration State': 'object', 'Plate Type': 'object',
 'Issue Date': 'object', 'Violation Code': 'int64', 'Vehicle Body Type': 'object', 'Vehicle Make': 'object',
 'Issuing Agency': 'object', 'Street Code1': 'int64', 'Street Code2': 'int64', 'Street Code3': 'int64',
 'Vehicle Expiration Date': 'int64', 'Violation Location': 'float64', 'Violation Precinct': 'int64', 'Issuer Precinct': 'int64',
 'Issuer Code': 'int64', 'Issuer Command': 'object', 'Issuer Squad': 'object', 'Violation Time': 'object',
 'Time First Observed': 'object', 'Violation County': 'object', 'Violation In Front Of Or Opposite': 'object', 'House Number': 'object',
 'Street Name': 'object', 'Intersecting Street': 'object', 'Date First Observed': 'int64', 'Law Section': 'int64',
 'Sub Division': 'object', 'Violation Legal Code': 'object', 'Days Parking In Effect    ': 'object', 'From Hours In Effect': 'object',
 'To Hours In Effect': 'object', 'Vehicle Color': 'object', 'Unregistered Vehicle?': 'float64', 'Vehicle Year': 'int64',
 'Meter Number': 'object', 'Feet From Curb': 'int64', 'Violation Post Code': 'object', 'Violation Description': 'object',
 'No Standing or Stopping Violation': 'float64', 'Hydrant Violation': 'float64', 'Double Parking Violation': 'float64'}

# This is to avoid the  DtypeWarning 
df = dd.read_csv(parking_violation_csv,  dtype=dict_1)
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,5092469481,GZH7067,NY,PAS,07/10/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
1,5092451658,GZH7067,NY,PAS,07/08/2016,7,SUBN,TOYOT,V,0,...,GY,NaN,2001,NaN,0,NaN,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN
2,4006265037,FZX9232,NY,PAS,08/23/2016,5,SUBN,FORD,V,0,...,BK,NaN,2004,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
3,8478629828,66623ME,NY,COM,06/14/2017,47,REFG,MITSU,T,10610,...,WH,NaN,2007,NaN,0,04,47-Double PKG-Midtown,NaN,NaN,NaN
4,7868300310,37033JV,NY,COM,11/21/2016,69,DELV,INTER,T,10510,...,WHITE,NaN,2007,NaN,0,31 6,69-Failure to Disp Muni Recpt,NaN,NaN,NaN


In [23]:
# Your code here
# Create a filtered Dask DataFrame with only the commercial plates.
df_filtered = df[df['Plate Type']=='COM']
# Persist it, so it is available in memory for future computations. 
df_filtered = df_filtered.persist()
df_2017 = df[df['Issue Date'].str.contains('2017')]

# df with number of summonses in 2017 (i.e., Issue Year in 2016, 2017) 
summonses_value_df = df_filtered[df_filtered['Issue Date'].str.contains('2017')]

# after reorganizing dataframe in one partition, check number of summonses
summonses_value_df_reduced =  summonses_value_df.repartition(npartitions=1)
num_commercial_2017 = summonses_value_df_reduced.map_partitions(len).compute()

#compute them as a percentage of the total data set
df_size = df_2017.index.size
pct_commercial = ((num_commercial_2017/df_size)*100)
num_commercial_2017 = int(num_commercial_2017)
pct_commercial = float(pct_commercial)

In [24]:
# Percentage relative to all the parking tickets in 2017
print(f'Number of NYC summonses with commercial plates in 2017 was {num_commercial_2017}')
print(f'Percentage {pct_commercial:5.2f}')

Number of NYC summonses with commercial plates in 2017 was 946620
Percentage 17.43
